In [1]:
from sklearn.tree import DecisionTreeRegressor

from sklearn.datasets import load_boston, load_iris
import numpy as np

In [2]:
from sklearn.preprocessing import OneHotEncoder

In [3]:
class MSE:
    def fit(self, y, yhat):
        return 0.5 * np.mean((y - yhat)**2)
    
    def gradient(self, y, yhat):
        return yhat - y
    
class CrossEntropyWithLogits:
    
    def softmax(self, x):
        c = np.max(x, axis=0)
        a = np.exp(x-c + 10e-4)
        sum_a = np.sum(a, axis=0)
        return a / sum_a
    
    def fit(self, y, logits):
        output = - np.mean(np.sum(y * np.log(self.softmax(logits)), axis=0))
        return output

    def gradient(self, y, logits):
        return logits - y

In [4]:
class GBDT:
    def __init__(self, n_estimator=3, learning_rate=0.01, regression=True, **params):
        self.n_estimator = n_estimator
        self.regression = regression
        self.learning_rate = learning_rate
        self.trees = [DecisionTreeRegressor(**params) for _ in range(self.n_estimator)]
        self.loss = MSE() if self.regression else CrossEntropyWithLogits()
        
    def fit(self, X, y):
        self.trees[0].fit(X, y)
        yhat = self.trees[0].predict(X)
        for i in range(1, self.n_estimator):
            negative_gradient = - self.loss.gradient(y, yhat)
            self.trees[i].fit(X, negative_gradient)
            yhat += self.learning_rate * self.trees[i].predict(X)
    
    def predict(self, X):
        yhat = self.trees[0].predict(X)
        for i in range(1, self.n_estimator):   
            yhat += self.learning_rate * self.trees[i].predict(X)
        return yhat

In [13]:
data = load_iris()

X = data['data']
y = data['target']
y = OneHotEncoder(sparse=False).fit_transform(y[:, np.newaxis])

params = {"max_depth": 3}
gbdt = GBDT(n_estimator=5, **params, regression=False)
gbdt.fit(X,y)

yhat = gbdt.predict(X)

acc =  np.mean(np.argmax(y, axis=0) == np.argmax(yhat, axis=0))
print("acc: ", acc)
for tree in gbdt.trees:
    print(tree.feature_importances_)

acc:  0.3333333333333333
[0.         0.         0.05393633 0.94606367]
[0.26777765 0.13675676 0.59546559 0.        ]
[0.26777765 0.67114896 0.06107339 0.        ]
[0.26777765 0.13675676 0.59546559 0.        ]
[0.26777765 0.67114896 0.06107339 0.        ]


In [10]:
data = load_boston()

X = data['data']
y = data['target']

params = {"max_depth": 10}
gbdt = GBDT(n_estimator=40, **params)
gbdt.fit(X,y)

yhat = gbdt.predict(X)
print((y - yhat)**2)

for tree in gbdt.trees:
    print(tree.feature_importances_)

[3.66722259e-06 1.58561636e-02 1.01148516e-02 3.92376675e-02
 6.40916518e-01 5.89601789e-03 3.47640364e+00 9.17678968e-04
 1.09886808e-03 3.04276439e-04 9.17678968e-04 5.50128663e-01
 0.00000000e+00 3.60149693e-02 1.53601592e+00 5.48720745e-02
 1.55464343e-01 4.74286149e-05 1.66642806e+00 1.29399884e-04
 4.54348180e-01 2.03384266e+00 4.83548363e-01 5.47810614e-02
 3.91599626e+00 1.98787289e-01 1.01422119e+00 3.61837157e-02
 1.79484644e+00 3.60836512e-03 8.57719051e-03 2.73049896e-03
 1.15749769e-01 2.08113262e+00 3.07607317e-01 4.51524889e+00
 1.34326841e+00 2.66625434e-01 4.69328275e+00 3.66722259e-06
 9.88580418e-03 2.77087835e-06 4.72425902e-01 6.90095470e-02
 5.20936233e-01 6.65168361e-01 1.24248015e-02 3.04276439e-04
 1.09886808e-03 2.45155435e-05 6.02270113e-02 4.79303985e-01
 3.80645155e-01 4.85046075e+00 0.00000000e+00 3.27892009e-07
 2.23931933e-01 3.66722259e-06 1.19458319e-03 1.26920133e-01
 3.03446406e-01 9.17678968e-04 1.18723484e-01 6.67104432e-02
 1.09676297e-02 2.738734

In [7]:
x = [[1], [2], [3]]
for i, item in enumerate(x):
    item = [0]
print(x)

[[1], [2], [3]]


In [8]:
x = [[1], [2], [3]]
for i, item in enumerate(x):
    item += [i]
print(x)

[[1, 0], [2, 1], [3, 2]]
